## PreProcess

In [87]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_from_disk

df = pd.read_csv('podcastdata_dataset.csv')
def preprocess_tolower(text):
    text=text.lower()
    return text

df['text']=df['text'].apply(preprocess_tolower)

## Splitter

In [88]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    # SemanticSplitterNodeParser,
)
from llama_index.core import Document 
# from llama_index.core.node_parser import SimpleNodeParser
# from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import LangchainNodeParser
# from llama_index.core.schema import MetadataMode 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [89]:
docs = [
    Document(
        text=row['text'],
        metadata={
            'Title': row['title'],
            'Guest': row['guest'],
            'id': row['id']
        }
    )
    for _, row in df[:3].iterrows()
]

parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap = 200
    )
)
nodes = parser.get_nodes_from_documents(docs)


In [98]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
test_emeds = embed_model.get_text_embedding("Hello World!")


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


d:\Projects\HYDE\.conda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


In [103]:
from llama_index.core import ServiceContext, set_global_service_context
ctx_base = ServiceContext.from_defaults(
    llm=None,
    embed_model=embed_model
)

C:\Users\sai18\AppData\Local\Temp\ipykernel_21060\2198921486.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  ctx_base = ServiceContext.from_defaults(


LLM is explicitly disabled. Using MockLLM.


In [106]:
from llama_index.core import VectorStoreIndex
sentence_index = VectorStoreIndex(nodes, service_context=ctx_base)

Batches: 100%|██████████| 1/1 [00:04<00:00,  4.77s/it]


In [110]:
print(sentence_index)